# GroupBy的实现

## 一、数据生成

In [1]:
import numpy as np
import pandas as pd

假设有14条记录

In [2]:
NUM = 14

In [3]:
np.random.seed(0)

### Party1: 拥有学号、姓名，年龄，年级，性别字段

In [4]:
id_list = np.arange(10000, 10000+NUM)

In [5]:
name_list = list('ABCDEFGHIGKLMN')

In [6]:
age_list = np.random.randint(10, 15, NUM)

In [7]:
grade_list = np.random.randint(0, 3, NUM)

In [8]:
sex_list = np.random.randint(0, 2, NUM)

In [9]:
data1 = pd.DataFrame({'id': id_list, 'name': name_list, 'grade': grade_list,
                      'sex': sex_list, 'age': age_list})

In [10]:
data1

,id,name,grade,sex,age
0,10000,A,2,0,14
1,10001,B,2,1,10
2,10002,C,0,0,13
3,10003,D,1,1,13
4,10004,E,1,1,13
5,10005,F,1,1,11
6,10006,G,1,1,13
7,10007,H,0,1,12
8,10008,I,1,0,14
9,10009,G,0,1,10


### Party2: 拥有班级、成绩

In [11]:
class_list = np.random.randint(1, 4, NUM)

In [12]:
score_list = np.random.randint(55, 99, NUM)

In [13]:
data2 = pd.DataFrame({'class': class_list, 'score': score_list})

In [14]:
data2

,class,score
0,1,72
1,3,70
2,1,59
3,3,96
4,3,97
5,1,86
6,3,56
7,1,56
8,1,94
9,1,96


## 二、GroupBy(实现方式一)

In [15]:
SELECTED = 'sex'

In [17]:
 data1.groupby(SELECTED)['age'].agg('min')

sex
0    10
1    10
Name: age, dtype: int64

## 1. 如果分组字段和聚合字段在同一数据方，直接进行聚合计算即可

In [175]:
data1['age_min'] = data1.groupby(SELECTED)['age'].transform('mean')

In [176]:
data1

,id,name,grade,sex,age,age_min
0,10000,A,2,0,14,12.75
1,10001,B,2,1,10,11.90
2,10002,C,0,0,13,12.75
3,10003,D,1,1,13,11.90
4,10004,E,1,1,13,11.90
5,10005,F,1,1,11,11.90
6,10006,G,1,1,13,11.90
7,10007,H,0,1,12,11.90
8,10008,I,1,0,14,12.75
9,10009,G,0,1,10,11.90


## 2. 如果分组字段和聚合字段在不同数据方

In [177]:
agg_col = 'score'

## 2.1 排序和记录索引

In [178]:
arg_list = data1[SELECTED].argsort()

In [179]:
selected_counts = data1[SELECTED].value_counts()

In [180]:
index = selected_counts.index.argsort()

In [181]:
counts = selected_counts[index].values

In [182]:
value = np.cumsum(counts)

In [183]:
index

array([1, 0])

按照索引分类取数据

In [184]:
value

array([10, 14])

In [185]:
value[0]

10

In [186]:
ids = arg_list[0: value[0]]

In [187]:
ids

0     0
1     2
2     8
3    10
4     1
5     3
6     4
7     5
8     6
9     7
Name: sex, dtype: int64

In [188]:
data2[agg_col].iloc[ids].max()

96

In [189]:
data2.loc[0:value[0],'max'] = 93

In [190]:
data2

,class,score,max
0,1,60,93.0
1,1,96,93.0
2,1,90,93.0
3,1,55,93.0
4,1,86,93.0
5,1,60,93.0
6,3,85,93.0
7,1,55,93.0
8,3,91,93.0
9,2,89,93.0


In [191]:
ids = arg_list[value[0]: value[1]]

In [192]:
ids

10     9
11    11
12    12
13    13
Name: sex, dtype: int64

In [193]:
data2.loc[value[0]:value[1],'max'] = 97

In [195]:
data2

,class,score,max
0,1,60,93.0
1,1,96,93.0
2,1,90,93.0
3,1,55,93.0
4,1,86,93.0
5,1,60,93.0
6,3,85,93.0
7,1,55,93.0
8,3,91,93.0
9,2,89,93.0


## 三、GroupBy(实现方式二)

### 1. 将原始数据中的内容替换掉，其实也就是保护起来

In [240]:
unique_list = data1[SELECTED].unique()

In [241]:
unique_list

array([0, 1])

In [242]:
unique_dict = dict(zip(unique_list, range(len(unique_list))))

In [243]:
unique_dict

{0: 0, 1: 1}

In [244]:
def applyFunc(x, unique_dict):
    return unique_dict[x]

In [245]:
data1[SELECTED]

0     0
1     1
2     0
3     1
4     1
5     1
6     1
7     1
8     0
9     1
10    0
11    1
12    1
13    1
Name: sex, dtype: int64

In [246]:
df = data1[SELECTED].apply(applyFunc, args=(unique_dict,))

In [247]:
df

0     0
1     1
2     0
3     1
4     1
5     1
6     1
7     1
8     0
9     1
10    0
11    1
12    1
13    1
Name: sex, dtype: int64

### 2. 另一个利用该数据，进行groupBy

In [248]:
data_new = pd.concat((data2, df), axis=1)

In [249]:
data2['score_mean'] = data_new.groupby(SELECTED)[agg_col].transform("mean")

In [250]:
data2

,class,score,score_mean
0,1,57,69.75
1,1,58,75.30
2,3,85,69.75
3,1,89,75.30
4,3,98,75.30
5,3,68,75.30
6,3,95,75.30
7,1,63,75.30
8,1,74,69.75
9,1,86,75.30
